# Lector Objetos de Tweets


### 1. Librerias

In [4]:
import zipfile
import json

### 2. Datos

#### 2.1. UnZip

Primero extraemos los datos de la carpeta .zip para poder leer los datos

In [3]:
DIR_ZIP = './data/archive.zip'
DIR_UNZIP = './data'

with zipfile.ZipFile(DIR_ZIP, 'r') as zip_ref:
    zip_ref.extractall(DIR_UNZIP)

#### 2.2 Data Lecture

In [ ]:
FILE_DIR = DIR_UNZIP + '/farmers-protest-tweets-2021-03-5.json'

with open(FILE_DIR) as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()